In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Uterine_Carcinosarcoma"
cohort = "GSE36133"

# Input paths
in_trait_dir = "../../input/GEO/Uterine_Carcinosarcoma"
in_cohort_dir = "../../input/GEO/Uterine_Carcinosarcoma/GSE36133"

# Output paths
out_data_file = "../../output/preprocess/Uterine_Carcinosarcoma/GSE36133.csv"
out_gene_data_file = "../../output/preprocess/Uterine_Carcinosarcoma/gene_data/GSE36133.csv"
out_clinical_data_file = "../../output/preprocess/Uterine_Carcinosarcoma/clinical_data/GSE36133.csv"
json_path = "../../output/preprocess/Uterine_Carcinosarcoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE36133_family.soft.gz', 'GSE36133_series_matrix.txt.gz']
Identified SOFT files: ['GSE36133_family.soft.gz']
Identified matrix files: ['GSE36133_series_matrix.txt.gz']



Background Information:
!Series_title	"Expression data from the Cancer Cell Line Encyclopedia (CCLE)"
!Series_summary	"The Cancer Cell Line Encyclopedia (CCLE) project is a collaboration between the Broad Institute, the Novartis Institutes for Biomedical Research and the Genomics Novartis Foundation to conduct a detailed genetic and pharmacologic characterization of a large panel of human cancer models"
!Series_summary	"It consists of a compilation of gene expression, chromosomal copy number, and massively parallel sequencing data from nearly 1,000 human cancer cell lines. All raw and processed data are available through an integrated portal on www.broadinstitute.org/ccle"
!Series_overall_design	"The final cell line collection spans 36 cancer types. Representation of cell lines for each cancer type was mainly driven by cancer mortality in the United States, as a surrogate of unmet medical need, as well as availability."

Sample Characteristics Dictionary:
{0: ['primary site: central_n

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This appears to be a cancer cell line expression dataset, so it's likely to contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For Uterine_Carcinosarcoma trait:
# Looking at primary site and histology keys
# Primary site key 0 has 'endometrium', which is relevant to uterine cancer
# But there's no specific indication of carcinosarcoma in the dictionary
# For Uterine Carcinosarcoma, we would need to combine information from multiple fields
trait_row = None  # Not directly available for this specific trait

# Age data
# There is no age information in the sample characteristics
age_row = None

# Gender data 
# No gender information is provided in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait values to binary format."""
    if value is None or pd.isna(value):
        return None
        
    # Extract the value after the colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # This function won't be used as trait_row is None, but defining it for completeness
    return None

def convert_age(value):
    """Convert age values to continuous format."""
    if value is None or pd.isna(value):
        return None
        
    # Extract the value after the colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # This function won't be used as age_row is None, but defining it for completeness
    return None

def convert_gender(value):
    """Convert gender values to binary format (0 for female, 1 for male)."""
    if value is None or pd.isna(value):
        return None
        
    # Extract the value after the colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # This function won't be used as gender_row is None, but defining it for completeness
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save cohort info with initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None, indicating clinical data is not available


False

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')

Gene expression data shape: (18926, 917)


### Step 4: Gene Identifier Review

In [5]:
# Reviewing gene identifiers in the gene expression data
# The identifiers follow the pattern: number_at
# These appear to be probe IDs from a microarray platform, not human gene symbols

# Based on biomedical knowledge, these identifiers (like '100009676_at', '10000_at') 
# are probe IDs from a microarray platform such as Affymetrix
# They need to be mapped to human gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
              GSM886835  GSM886836  GSM886837  GSM886838  GSM886839
ID                                                                 
100009676_at     6.1161     6.2052     6.1249     6.6154     5.4236
10000_at         8.1556     6.6152     4.5676     4.3519     6.6723
10001_at         9.7864     9.9699     8.8720     9.1376    10.0290
10002_at         3.7977     4.0304     3.8455     3.7085     3.6431
10003_at         3.5458     3.8504     4.0458     3.9508     4.1589



Platform information:
!Series_title = Expression data from the Cancer Cell Line Encyclopedia (CCLE)
!Platform_title = Affymetrix Human Genome U133 Plus 2.0 Array [Brainarray Version 15.0.0, HGU133Plus2_Hs_ENTREZG]
!Platform_description = This array is identical to GPL570 but the data were analyzed with a custom CDF Brainarray Version 15, hgu133plus2hsentrezg.
#Description =
ID	ORF	Description
!Sample_description = Gene expression data from the CCLE
!Sample_description = Gene expression data from the CCLE
!Sample_description = Gene expression data from the CCLE
!Sample_description = Gene expression data from the CCLE
!Sample_description = Gene expression data from the CCLE
!Sample_description = Gene expression data from the CCLE
!Sample_description = Gene expression data from the CCLE
!Sample_description = Gene expression data from the CCLE
!Sample_description = Gene expression data from the CCLE
!Sample_description = Gene expression data from the CCLE
!Sample_description = Gene expres


Gene annotation columns:
['ID', 'ORF', 'Description']

Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ORF': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}



Matching rows in annotation for sample IDs: 9180

Potential gene symbol columns: []

Is this dataset likely to contain gene expression data? True


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# From the preview, we can see that:
# - The ID column in gene_annotation contains probe IDs like '1_at', which match the format in gene_data
# - The ORF column contains what appear to be Entrez IDs
# - The Description column contains gene names/descriptions

# Let's extract the mapping between probe IDs and gene symbols (from Description field)
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Description')

# Preview the mapping to confirm it looks correct
print("Gene mapping preview:")
print(gene_mapping.head())
print(f"Gene mapping shape: {gene_mapping.shape}")

# 2. Apply the mapping to convert probe-level measurements to gene expression data
# The library function apply_gene_mapping handles many-to-many relationships as described
gene_data_mapped = apply_gene_mapping(gene_data, gene_mapping)

# Rename to meet the requirement
gene_data = gene_data_mapped

# Preview the mapped gene data
print("\nMapped gene data preview:")
print(gene_data.head())
print(f"Gene data shape after mapping: {gene_data.shape}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nSaved mapped gene expression data to {out_gene_data_file}")


Gene mapping preview:
         ID                                               Gene
0      1_at                             alpha-1-B glycoprotein
1     10_at  N-acetyltransferase 2 (arylamine N-acetyltrans...
2    100_at                                adenosine deaminase
3   1000_at          cadherin 2, type 1, N-cadherin (neuronal)
4  10000_at  v-akt murine thymoma viral oncogene homolog 3 ...
Gene mapping shape: (18909, 2)



Mapped gene data preview:
      GSM886835  GSM886836  GSM886837  GSM886838  GSM886839  GSM886840  \
Gene                                                                     
A-    39.130283  40.260633  38.686433  37.203967  40.561633  39.573033   
A-2    4.666800   4.545600   4.542000   6.251500   4.606400   5.234100   
A-52  13.361000  12.996000  13.073000  13.024000  13.232000  13.260000   
A-I   14.850300  13.204300  15.684000  15.212200  13.680400  14.789400   
A-II   3.817800   3.682100   3.711500   3.778300   3.757900   4.049600   

      GSM886841  GSM886842  GSM886843  GSM886844  ...  GSM887742  GSM887743  \
Gene                                              ...                         
A-    40.203917   38.98245   42.64055   40.24315  ...   39.49325   38.85855   
A-2    4.935200    4.99450    5.02700    4.94430  ...    4.73400    4.97930   
A-52  13.453000   12.83100   13.21000   13.07200  ...   13.48000   12.37200   
A-I   13.883200   15.05300   14.29620   14.39730  ...   14.


Saved mapped gene expression data to ../../output/preprocess/Uterine_Carcinosarcoma/gene_data/GSE36133.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Try to use the provided normalization function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
except Exception as e:
    print(f"Error during gene symbol normalization: {e}")
    # Fall back to using the gene data as is if normalization fails
    normalized_gene_data = gene_data
    print(f"Using original gene data: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. We need to handle the case where trait_row is None
if trait_row is None:
    # Create a minimal DataFrame with the same structure expected by the validation function
    # This ensures we have a non-empty DataFrame with appropriate structure for validation
    sample_ids = normalized_gene_data.columns[:5]  # Get a few sample IDs
    dummy_df = pd.DataFrame({
        trait: [None] * len(sample_ids)
    }, index=sample_ids)
    
    # No clinical features to extract
    clinical_features = pd.DataFrame()
    print("No clinical features available for this dataset.")
    
    # Create a minimal dummy DataFrame for linked data
    linked_data = dummy_df
    
    # For datasets without trait data, mark them as biased (since there's no trait distribution at all)
    is_trait_biased = True
    
    # Validate and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,  # We have no trait data
        is_biased=is_trait_biased,  # Must provide a boolean value, not None
        df=linked_data,
        note="Dataset contains CCLE gene expression data but lacks specific Uterine_Carcinosarcoma annotations."
    )
    
    print(f"Data quality check result: Not usable (No trait data)")
else:
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_data, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    
    # 4. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    
    # 6. Conduct quality check and save the cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True,  
        is_trait_available=True,
        is_biased=is_trait_biased, 
        df=linked_data,
        note="Cancer Cell Line Encyclopedia (CCLE) gene expression data"
    )
    
    # 7. Save the linked data if it's usable
    print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
    if is_usable:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (1266, 917)


Gene data saved to ../../output/preprocess/Uterine_Carcinosarcoma/gene_data/GSE36133.csv
No clinical features available for this dataset.
Abnormality detected in the cohort: GSE36133. Preprocessing failed.
Data quality check result: Not usable (No trait data)
